In [ ]:
# Importa a gdal
from osgeo import gdal, ogr, osr
gdal.UseExceptions()
from gdalconst import *
# Importa o matplotlib
import matplotlib.pyplot as plt
# Importa o Numpy
import numpy as np

# Carrega a máscara de nuvens

In [ ]:
# Abre a mascara de nuvem gerada anteriormente
nome_arquivo_mascara_nuvens = '../imagens/mascara_nuvens_CBERS_20180131.tif'

print ("Abrindo o arquivo: " + nome_arquivo_mascara_nuvens)

# Tenta abrir a imagem
try:
    mascara_nuvens = gdal.Open(nome_arquivo_mascara_nuvens, GA_ReadOnly)
    print("Arquivo aberto com sucesso!")
except:
    print("Erro na abertura do arquivo!")

In [ ]:
# Obtém a banda única do dataset
banda_mascara = mascara_nuvens.GetRasterBand(1)
# Transforma a banda em array
array_banda_mascara = banda_mascara.ReadAsArray()

# Gera a máscara de sombras inicial (vetorização da máscara de nuvens)

Aqui foi feita a vetorização da máscara de nuvens proveniente do outro notebook. A falta de compatibilidade entre as cenas é pq não tem um src atrelado à mascara ainda, a Grazi já ajeitou isso no git dela. Vamos atualizar assim que decidimos os commit kk

In [ ]:
# import sys (Irei tirar isso aqui, foi usado só na hora dos testes. Na revisão final vemos a real necessidade deles)
# import os

dst_layername = '../imagens/mascara_sombra'
drv = ogr.GetDriverByName("ESRI Shapefile")
    
dst_ds = drv.CreateDataSource( dst_layername + ".shp" )
dst_layer = dst_ds.CreateLayer(dst_layername)

#newField = ogr.FieldDefn('MYFLD', ogr.OFTInteger)
#dst_layer.CreateField(newField)

gdal.Polygonize( banda_mascara, banda_mascara, dst_layer, -1, callback=None )

# Deslocamento da máscara de nuvens a partir de um delta_x e delta_y genérico. 

Nesse caso foi usado um delta = (1,1). Ainda estou desenvolvendo a extração desses valores a partir dos metadados

In [ ]:
import fiona

# Essas três funções realizam o deslocamento das feições da camada. (Elas são recursivas entre si)
def shiftCoords_Point(coords, delta):     # "delta" é uma tupla (delta_x, delta_y)
    return tuple(c + d for c, d in zip(coords, delta))
def shiftCoords_LineString(coords, delta):
    return list(shiftCoords_Point(pt_coords, delta) for pt_coords in coords)
def shiftCoords_Polygon(coords, delta):
    return list(shiftCoords_LineString(ring_coords, delta) for ring_coords in coords)

# Aqui foi definido um valor genérico. 
delta = (1.0, 1.0) # As funções que farão a extração das informações dos metadados vão alterar exatamente aqui

with fiona.open('../imagens/mascara_sombra.shp', "r") as source:

    # Create a sink for processed features with the same format and 
    # coordinate reference system as the source.
    with fiona.open("../imagens/shifted.shp", 
                    "w",
                    driver=source.driver,
                    schema=source.schema,
                    crs=source.crs) as sink:
        # Deslocamento de todas as feições da camada de máscara de nuvens vetorizada
        for rec in source:
            try:
                rec['geometry']['coordinates'] = shiftCoords_Polygon(rec['geometry']['coordinates'], delta)
                sink.write(rec)
            except:
                print("Error processing record %s:", rec)
            

# Subtração da mascara deslocada pela mascara de nuvens 

Onde é nuvem não é sombra, então será removida a interseção entre as duas camadas. Assim, permanece na mascara exclusivamente o que é sombra

In [ ]:
from shapely.geometry import shape, mapping

with fiona.open('../imagens/mascara_sombra.shp', "r") as source:
    with fiona.open("../imagens/shifted.shp", "r") as shift:
        
        with fiona.open("../imagens/shifted_difference.shp", 
                    "w",
                    driver=source.driver,
                    schema=source.schema,
                    crs=source.crs) as dif:
            
            for i, j in zip(shift,source):
                
                # Corrigir erros topológico que impedem a subtração dos poligonos                
                polygons_source = shape(j['geometry'])
                polygons_shift = shape(i['geometry'])
                
                polygons_source_correct = polygons_source.buffer(0)
                polygons_shift_correct = polygons_shift.buffer(0)
                
                j['geometry'] = mapping(polygons_source_correct)
                i['geometry'] = mapping(polygons_shift_correct)
                #################################################################
            
                try:
                    difference = shape(i['geometry']).difference(shape(j['geometry']))
                    i['geometry'] = mapping(difference)
                    dif.write(i)
                except:
                    print("Error processing record %s:", i)
                

            

# A partir daqui estou desenvolvendo ainda..

In [ ]:
import math

sun_azim = math.radians(67.9437)
sun_zenith = math.radians(56.0513)
view_azim = math.radians(0)
view_zenith = math.radians(90)

shadow_direction = math.pi + math.atan(math.sin(sun_azim)*math.tan(sun_zenith)-math.sin(view_azim)*math.tan(view_zenith)/(math.cos(sun_azim)*math.tan(sun_zenith)-math.cos(view_azim)*math.tan(view_zenith)))
print(shadow_direction)